## 資料處理-TOKYO

In [25]:
import os
import pandas as pd
import re

# -------------------------------
# 共同使用的函數
# -------------------------------

# 分類出發時間
def classify_departure_time_period(departure_time):
    if "凌晨" in departure_time or "清晨" in departure_time:
        hour = int(departure_time.split("清晨")[-1].split(":")[0]) if "清晨" in departure_time else int(departure_time.split("凌晨")[-1].split(":")[0])
        if hour == 12 or 0 <= hour < 6:  # 特別處理凌晨 12:xx 和 0:xx
            return "凌晨班機"
    if "清晨" in departure_time or "早上" in departure_time or "上午" in departure_time:
        hour = int(departure_time.split(":")[0].split("清晨")[-1].split("早上")[-1].split("上午")[-1])
        if 6 <= hour < 9:
            return "早晨班機"
        elif 9 <= hour < 12:
            return "上午班機"
    if "中午" in departure_time:
        hour = int(departure_time.split("中午")[-1].split(":")[0])
        if hour == 12:
            return "下午班機"
    if "下午" in departure_time:
        hour = int(departure_time.split("下午")[-1].split(":")[0]) + 12
        if 12 <= hour < 18:
            return "下午班機"
    if "晚上" in departure_time or "下午" in departure_time:
        hour = int(departure_time.split("晚上")[-1].split("下午")[-1].split(":")[0]) + 12
        if 18 <= hour < 24:
            return "晚間班機"
    return "未分類"

# 分類抵達時間
def classify_arrival_time_period(arrival_time):
    if "凌晨" in arrival_time or "清晨" in arrival_time:
        hour = int(arrival_time.split("清晨")[-1].split(":")[0]) if "清晨" in arrival_time else int(arrival_time.split("凌晨")[-1].split(":")[0])
        if hour == 12 or 0 <= hour < 6:
            return "凌晨抵達"
    if "清晨" in arrival_time or "早上" in arrival_time or "上午" in arrival_time:
        hour = int(arrival_time.split(":")[0].split("清晨")[-1].split("早上")[-1].split("上午")[-1])
        if 6 <= hour < 9:
            return "早晨抵達"
        elif 9 <= hour < 12:
            return "上午抵達"
    if "中午" in arrival_time:
        hour = int(arrival_time.split("中午")[-1].split(":")[0])
        if hour == 12:
            return "下午抵達"
    if "下午" in arrival_time:
        hour = int(arrival_time.split("下午")[-1].split(":")[0]) + 12
        if 12 <= hour < 18:
            return "下午抵達"
    if "晚上" in arrival_time or "下午" in arrival_time:
        hour = int(arrival_time.split("晚上")[-1].split("下午")[-1].split(":")[0]) + 12
        if 18 <= hour < 24:
            return "晚間抵達"
    return "未分類"

# 剔除離群值（依需求啟用）
def remove_outliers(df, column):
    """剔除指定欄位中超出平均值三倍標準差範圍的離群值"""
    mean = df[column].mean()
    std = df[column].std()
    lower_bound = mean - 3 * std
    upper_bound = mean + 3 * std
    clean_df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    print(f"剔除離群值前資料筆數：{len(df)}, 剔除離群值後資料筆數：{len(clean_df)}")
    return clean_df

# 分類機型：0 窄體機, 1 寬體機, 2 超大型客機, 其他 -1
def classify_aircraft_type(aircraft_model):
    if pd.isnull(aircraft_model):
        return -1
    elif any(x in aircraft_model for x in ['Airbus A320', 'Airbus A321neo', 'Airbus A320neo', 'Boeing 737']):
        return 0  # 窄體機
    elif any(x in aircraft_model for x in ['Boeing 787', 'Boeing 787-10', 'Airbus A330', 'Airbus A330-900neo', 'Airbus A350', 'Boeing 777', 'Boeing 767']):
        return 1  # 寬體機
    elif any(x in aircraft_model for x in ['Airbus A380', 'Boeing 747']):
        return 2  # 超大型客機
    else:
        return -1

# -------------------------------
# 主處理與合併函數
# -------------------------------
def process_and_merge_files(start_date, end_date, base_path, output_path, file_pattern):
    """
    Parameters:
      start_date: 字串，例如 '1021' 表示 MMDD
      end_date: 字串，例如 '0320'
      base_path: 資料所在目錄
      output_path: 輸出 CSV 的完整路徑
      file_pattern: 檔案名稱格式，內含 {} 以填入日期字串，
                    例如 "tokyo_{}.csv" 或 "tokyo_business_{}.csv"
    """
    # 航空聯盟分類（精簡版）
    star_alliance = ['長榮航空', '全日空航空']
    skyteam = ['中華航空']
    oneworld = ['日本航空', '國泰航空']
    value_alliance = ['酷航']
    
    # 建立日期範圍
    dates = pd.date_range(start=f'2024-{start_date[:2]}-{start_date[2:]}', 
                          end=f'2025-{end_date[:2]}-{end_date[2:]}')
    date_strings = [date.strftime('%m%d') for date in dates]

    data_frames = []
    for date in date_strings:
        file_path = f'{base_path}/{file_pattern.format(date)}'
        if os.path.exists(file_path):
            # 從檔名推斷檔案日期
            month, day = int(date[:2]), int(date[2:])
            year = 2024 if month >= 10 else 2025
            file_date = pd.Timestamp(year=year, month=month, day=day)
            df = pd.read_csv(file_path)
            if not df.empty:
                df = df.dropna(how="all")
                
                # 格式化出發日期並去除時間
                df['出發日期'] = pd.to_datetime(df['出發日期'].str.extract(r'(\d{4}-\d{2}-\d{2})')[0]).dt.strftime('%Y-%m-%d')
                # 新增星期欄位
                df['星期'] = pd.to_datetime(df['出發日期']).dt.day_name(locale='zh_TW')
                
                # 移除價格中的符號並轉換為數字
                df = df[df['價格'].str.match(r'^[NT\$,\d\s]+$')]
                df['價格'] = df['價格'].replace(r'[NT\$,\s]', '', regex=True).astype(int)
                
                # 格式化出發日期並計算 day left
                df['出發日期'] = pd.to_datetime(df['出發日期'])
                df['day left'] = (df['出發日期'] - file_date).dt.days
                
                # 新增航空聯盟欄位
                def assign_alliance(airline):
                    if airline in star_alliance:
                        return 1
                    elif airline in skyteam:
                        return 2
                    elif airline in oneworld:
                        return 3
                    elif airline in value_alliance:
                        return 4
                    elif airline in ['星宇航空']:
                        return 5  # 無聯盟傳統航空
                    elif airline in ['樂桃航空', '台灣虎航', '捷星日本航空', '泰國獅航']:
                        return 6  # 無聯盟廉價航空
                    else:
                        return 0  # 其他或無法分類
                
                df['航空聯盟'] = df['航空公司'].apply(assign_alliance)
                
                # 調整欄位順序：將航空聯盟插入航空公司之後
                cols = df.columns.tolist()
                if "航空公司" in cols and "航空聯盟" in cols:
                    cols.insert(cols.index("航空公司") + 1, cols.pop(cols.index("航空聯盟")))
                df = df[cols]
                # 將星期欄位移到出發日期後面
                cols = df.columns.tolist()
                if "出發日期" in cols and "星期" in cols:
                    cols.insert(cols.index("出發日期") + 1, cols.pop(cols.index("星期")))
                df = df[cols]
                
                # 新增出發時段欄位
                df['出發時段'] = df['出發時間'].apply(classify_departure_time_period)
                cols = df.columns.tolist()
                if '出發時間' in cols and '出發時段' in cols:
                    cols.insert(cols.index('出發時間') + 1, cols.pop(cols.index('出發時段')))
                df = df[cols]
                
                # 新增抵達時段欄位
                df['抵達時段'] = df['抵達時間'].apply(classify_arrival_time_period)
                cols = df.columns.tolist()
                if '抵達時間' in cols and '抵達時段' in cols:
                    cols.insert(cols.index('抵達時間') + 1, cols.pop(cols.index('抵達時段')))
                df = df[cols]
                
                data_frames.append(df)

    merged_data = pd.concat(data_frames, ignore_index=True) if data_frames else pd.DataFrame()
    
    if merged_data.empty:
        print("沒有合併到任何資料。")
        return
    
    # 新增「機型分類」欄位
    merged_data['機型分類'] = merged_data['機型'].apply(classify_aircraft_type)
    cols = merged_data.columns.tolist()
    if '機型分類' in cols and '機型' in cols:
        cols.insert(cols.index('機型') + 1, cols.pop(cols.index('機型分類')))
    merged_data = merged_data[cols]

    # 計算每個航班的筆數
    flight_counts = merged_data.groupby(['出發日期', '航班代碼']).size().reset_index(name='筆數')

    # 統計有多少筆被剔除
    original_flight_count = flight_counts.shape[0]
    valid_flights = flight_counts[flight_counts['筆數'] >= 35]
    removed_flight_count = original_flight_count - valid_flights.shape[0]
    print(f"原始航班數量：{original_flight_count}，刪除少於 35 筆的航班數量：{removed_flight_count}")

    # 僅保留符合條件的航班資料
    merged_data = pd.merge(merged_data, valid_flights[['出發日期', '航班代碼']], 
                        on=['出發日期', '航班代碼'], how='inner')

    # 分組計算價格統計數據
    grouped = merged_data.groupby(['出發日期', '航班代碼'], as_index=False).agg(
        平均價格=('價格', 'mean'),
        最低價格=('價格', 'min'),
        中位數價格=('價格', 'median'),
        價格變異=('價格', 'var')
    )
    grouped['平均價格'] = grouped['平均價格'].round(2).astype(int)
    grouped['最低價格'] = grouped['最低價格'].round(2).astype(int)
    grouped['中位數價格'] = grouped['中位數價格'].round(2).astype(int)
    grouped['價格變異'] = grouped['價格變異'].fillna(0).round(2)
    
    # 新增「隨機購買平均價格」：對每一組隨機抽取 4 筆價格並計算平均
    grouped_random = merged_data.groupby(['出發日期', '航班代碼'])['價格'] \
        .apply(lambda x: round(x.sample(4).mean(), 2)).reset_index(name='隨機購買平均價格')

    # 合併隨機購買平均價格到 grouped
    grouped = pd.merge(grouped, grouped_random, on=['出發日期', '航班代碼'], how='inner')
 
    # 取出最低價格的剩餘天數（原來的做法）
    min_price_days = merged_data.loc[
        merged_data.groupby(['出發日期', '航班代碼'])['價格'].idxmin(),
        ['出發日期', '航班代碼', 'day left']
    ]
    min_price_days.rename(columns={'day left': '最低價格剩餘天數'}, inplace=True)
    
    # 合併分組後的統計數據
    final_data = pd.merge(merged_data.drop(columns=['價格', 'day left']),
                          grouped, on=['出發日期', '航班代碼'], how='inner')
    final_data = pd.merge(final_data, min_price_days, on=['出發日期', '航班代碼'], how='inner')
    final_data = pd.merge(final_data, valid_flights, on=['出發日期', '航班代碼'], how='inner')

    # 去除重複行，只保留同一航班的最後一筆資料
    final_data = final_data.sort_values(by=['出發日期', '航班代碼', '筆數']).drop_duplicates(
        subset=['出發日期', '航班代碼'], keep='last'
    )
    final_data = final_data.sort_values(by=['出發日期', '航班代碼']).reset_index(drop=True)

    # (可選) 如需剔除「平均價格」超出三倍標準差的離群值，可啟用下行程式
    # final_data = remove_outliers(final_data, '平均價格')
    
    final_data.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"處理完成，結果已剔除少於 35 筆的航班，並儲存到 {output_path}")

# -------------------------------
# 主程式執行流程
# -------------------------------

# 設定基本路徑（請依據實際資料位置修改）
base_path = '/Users/yuchingchen/Documents/專題/web_scrapying/web_scrapying_data'

# 1. 處理一般班資料（檔案格式：tokyo_{MMDD}.csv）
output_path_tokyo = '/Users/yuchingchen/Documents/專題/merge_and_cleaned/data/short/tokyo.csv'
process_and_merge_files('1021', '0320', base_path, output_path_tokyo, file_pattern="tokyo_{}.csv")

# 2. 處理商務艙資料（檔案格式：tokyo_business_{MMDD}.csv）
output_path_tokyo_business = '/Users/yuchingchen/Documents/專題/merge_and_cleaned/data/short/tokyo_business.csv'
process_and_merge_files('1021', '0320', base_path, output_path_tokyo_business, file_pattern="tokyo_business_{}.csv")

原始航班數量：1939，刪除少於 35 筆的航班數量：63
處理完成，結果已剔除少於 35 筆的航班，並儲存到 /Users/yuchingchen/Documents/專題/merge_and_cleaned/data/short/tokyo.csv
原始航班數量：1237，刪除少於 35 筆的航班數量：26
處理完成，結果已剔除少於 35 筆的航班，並儲存到 /Users/yuchingchen/Documents/專題/merge_and_cleaned/data/short/tokyo_business.csv
